In [ ]:
-- functors apply a function to a boxed value
-- applicatives apply a boxed function to a boxed value
-- monads apply a function that returns a boxed value to a boxed value

-- supose we have a function half that only worked on even numbers
half :: (Integral a) => a -> Maybe a
half x | even x    = Just $ x `div` 2
       | otherwise = Nothing
       
half 10
half 11

-- if we want to operate on a boxed value we can use the >>= operator
Just 10 >>= half
Just 11 >>= half

-- Monad is defined as

-- class Monad m where
--     (>>=) :: m a -> (a -> m b) -> m b

-- the >>= operator takes a monad
-- and a function that returns a monad
-- and returns a monad (after applying the function to the monad)

In [ ]:
-- infact the do notation that we used before is just syntax sugar
-- do can combine any monad

-- foo = do
--     filename <- getLine
--     contents <- readFile filename
--     putStrLn contents
    
-- this is just syntax sugar for:

-- getLine >>= readFile >>= putStrLn

-- here is an example
foo :: Maybe String
foo = do
    x <- Just 3
    y <- Just "!"
    Just (show x ++ y)
foo

-- the alternate would be:
bar = Just 3 >>= (\x -> Just "!" >>= (\y -> Just (show x ++ y)))  
bar

-- the do will wrap each of the lines in a lambda 
-- and use the bind operator to combine them all

In [ ]:
-- lists are also monads

-- instance Monad [] where
--     return x = [x]
--     xs >>= f = concat (map f xs)
--     fail _ = []
    
-- return is the same consept as pure in Applicatives
-- the bind operator applys the function to the list
-- then concatinates it to the list of results
-- the fail function is equivilent to Nothing

In [ ]:
-- monads that also act as monoids are monda plus

-- class Monad m => MonadPlus m where  
--     mzero :: m a  
--     mplus :: m a -> m a -> m a

-- mzero is the same as mempty from Monoid
-- mplus is the same as mappend from Monoid

-- the guard function is defined as such

-- guard :: (MonadPlus m) => Bool -> m ()  
-- guard True = return ()  
-- guard False = mzero

-- this allows us to filter MonadPlus's as if we were doing list comprehensions
do  
    x <- [1..50]  
    guard ('7' `elem` show x)  
    return x 
    
-- which we could have writter like
[x | x <- [1..50], '7' `elem` show x]

In [ ]:
 -- Given a start position, can a knight to an end position in three moves
import Control.Monad

-- first we define a type for the knights position 
type Knight = (Int, Int)

-- next we need a function that returns a list of all the next moves
moveKnight :: Knight -> [Knight]
moveKnight (col,row) = do
    (col', row') <- [(col+2,row-1),(col+2,row+1),(col-2,row-1),(col-2,row+1),
                    (col+1,row-2),(col+1,row+2),(col-1,row-2),(col-1,row+2)]  
    guard (col' `elem` [1..8] && row' `elem` [1..8])
    return (col', row')
    
moveKnight (6, 2)

-- next we need to repeat this for each move 3 times
move3 :: Knight -> [Knight]
move3 start = moveKnight start >>= moveKnight >>= moveKnight

move3 (6, 2)

-- finaly we can define a function that tells us if we can reach a position in 3 moves
canReachIn3 :: Knight -> Knight -> Bool
start `canReachIn3` end = end `elem` move3 start

(6,2) `canReachIn3` (6,1)
(6,2) `canReachIn3` (7,3)